# AFMFIT TUTORIAL version 0.1

## 1. Import the AFM images

The first step is to import the AFM images 

In [1]:
%matplotlib
from afmfit.image import ImageSet

# import the example images contained in the tiff
imgs = ImageSet.read_tif("/home/AD/vuillemr/Code/afm-fitting/examples/examples_data/ef2.tiff", vsize=7.0, unit="m")

# displays the images in a separate window
imgs.show()

Using matplotlib backend: <object object at 0x7f217a231ed0>
Read 100 images of size 40 x 40 


## 2. Mask the input images

It is useful to mask the contour of the molecule to facilitate the fitting

In [2]:
from afmfit.image import mask_interactive

# The default parameters can be kept
masks = mask_interactive(imgs.imgs)

## 3. Import the PDB

Read the PDB file to be used as initial model 

In [3]:
from afmfit.pdbio import PDB

# read the PDB file
pdb = PDB( "/home/AD/vuillemr/Code/afm/ref.pdb")

# center the atoms around the coordinate 0,0,0
pdb.center()

print("The PDB file contains %i atoms"%pdb.n_atoms)


The PDB file contains 6215 atoms


## 4. Calculate NMA

Performs Normal Mode Analysis using NOLB

In [4]:
from afmfit.nma import NormalModesRTB

# Set the desired number of modes and cutoff (in Angstrom) for the elastic network model (higher cutoff= more rigid)
nma = NormalModesRTB.calculate_NMA(pdb,nmodes=3, cutoff=8.0, tmpDir="/home/AD/vuillemr/Code/afm/tmp")


*******************************************************************
*-----------------------------------------------------------------*
*-----------NOLB : a Non-Linear rigid block NMA method------------*
*----------Authors: Alexandre Hoffmann & Sergei Grudinin----------*
*---------Copyright (c): UGA/CNRS Grenoble, France, 2023.---------*
*-------- e-mail: sergei.grudinin@univ-grenoble-alpes.fr ---------*
*---- http://team.inria.fr/nano-d/software/nolb-normal-modes/ ----*
*-----------------------------------------------------------------*
*******************************************************************
Hostname......................................................... : ljk-ad-77
Run at........................................................... : Fri Apr  5 15:34:56 2024
Command-line arguments........................................... : /home/AD/vuillemr/NOLB/NOLB /home/AD/vuillemr/Code/afm/tmp.pdb -o /home/AD/vuillemr/Code/afm/tmp -s 0 -n 3 --format 3 -c 8.000000 
==================

## 5. Set the AFM image simulator

The fitting compares AFM-like images of the initial model to AFM experiments.
This section defines the parameters of the AFM simulator.

In [27]:
%matplotlib
from afmfit.simulator import AFMSimulator
from afmfit.viewer import viewAFM

# Set the parameters to simulate the pseudo-AFM images used for the fitting
#  - sigma controls the smoothness
#  - cutoff (angstrom) decrease the computational time for smaller values 
#  - beta can be kept to 1
simulator = AFMSimulator(size=imgs.size, 
                         vsize=imgs.vsize, 
                         beta=1.0, sigma=3.2, cutoff=40)

# Create an image of the PDB that have been loaded
image = simulator.pdb2afm(pdb)

# show the generated image
viewAFM(image)


Using matplotlib backend: TkAgg


(<Figure size 500x400 with 2 Axes>, <Axes: xlabel='nm', ylabel='nm'>)

## 6. Rigid Fitting

The rigid fitting performs a global alignment of the PDB with the images

In [5]:
# import the fitter
from afmfit.fitting import Fitter

In [32]:
import numpy as np

# defines the fitter
fitter = Fitter(pdb=pdb, imgs=masks * imgs.imgs, simulator=simulator)

# Performs the rigid fitting
# - n_cpu : set the number of cpus to use
# - angular_dist :  set the angular increment for the rigid-body search
# - zshift_range : set the translational increment in the z-axis for the rigid body search. Ex : between -20 Ang and 20 Ang with 10 increments
fitter.fit_rigid(n_cpu=12, 
                 angular_dist=10, 
                 verbose=True, 
                 zshift_range=np.linspace(-20,20,10))

# save the fitting on the disk
fitter.dump("/home/AD/vuillemr/Code/afm/rigid.pkl")

Projection Matching: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [01:18<00:00,  1.28it/s]


In [6]:
# load the fitting
fitter = Fitter.load("/home/AD/vuillemr/Code/afm/rigd.pkl")

# show the angular assignement
fitter.show_rigid()

FileNotFoundError: [Errno 2] No such file or directory: '/home/AD/vuillemr/Code/afm/rigd.pkl'

## 7. Flexible fitting

Flexibly aligns the model with the images

In [34]:
fitter.fit_flexible(n_cpu=12, 
                    nma=nma, 
                    n_best_views=5,
                    n_iter=10, 
                    gamma=20, 
                    gamma_rigid=10, 
                    verbose=True)

fitter.dump("/home/AD/vuillemr/Code/afm/flexible.pkl")

Flexible Fitting: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:28<00:00,  2.09s/it]


In [35]:
fitter = Fitter.load("/home/AD/vuillemr/Code/afm/flexible.pkl")

fitter.show()

## 8 Conformational Space with PCA

In [37]:
from afmfit.utils import DimRed

pca =DimRed.from_fitter(fitter=fitter, method="pca", n_components=10, )
# umap =DimRed.from_fitter(fitter=fitter, method="umap", n_components=2 )

pca.show(ax=[0,1])
pca.show_pca_ev()